<a href="https://colab.research.google.com/github/haromero/DAI-Distributed-Artifical-Inteligence-/blob/main/General_CVRP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROYECTO CVRP (SWARM INTELLIGENCE)**

# Librerías

In [ ]:
# Manejo de datos
import pandas as pd
import numpy as np

# Archivos y sistema
import os
import math
import random

# Clustering
from sklearn.cluster import KMeans
from typing import List, Tuple

# Cálculo de distancias
from scipy.spatial import distance_matrix

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx



# Carga de datos

## Problemset

In [ ]:
# Cargar archivo
df = pd.read_excel("/content/problemset.xlsx")

styled_df = df.style.set_properties()
styled_df = df.style.set_properties()
pd.set_option('display.max_colwidth', None)  # Mostrar todo el contenido de la celda
pd.set_option('display.expand_frame_repr', False)  # Evitar que se corte en varias líneas

# Add styling to the header and cells
styled_df = styled_df.set_table_styles([
    {'selector': 'th',  # Selects all <th> elements (header cells)
     'props': [('background-color', 'lavender'),  # Set background color
               ('font-weight', 'bold'),             # Set font weight to bold
               ('font-size', '13pt'),               # Set font size
               ('text-align', 'center'),
               ('border', '1px solid black')]       # Add a border
    },
    {'selector': 'td',  # Selects all <td> elements (data cells)
     'props': [('border', '1px solid black')]       # Add a border
    }
])

print(df.columns)
display(styled_df)
#print(df.head())
#print(df)

Index(['problem_cluster', 'nodes', 'distance_matrix', 'vehicle_capacity',
       'bf best route', 'bf best value', 'bf n vehicles', 'cw best route',
       'cw best value', 'cw n vehicles', 'aco best route', 'aco best value',
       'aco n vehicles'],
      dtype='object')


,problem_cluster,nodes,distance_matrix,vehicle_capacity,bf best route,bf best value,bf n vehicles,cw best route,cw best value,cw n vehicles,aco best route,aco best value,aco n vehicles
0,small-10n-c50_70-d10_50,problemset/in/small-10n-c50_70-d10_50_nodes.parquet,problemset/in/small-10n-c50_70-d10_50_dm.parquet,50,"[array([1, 2], dtype=int64) array([4, 5], dtype=int64) array([6], dtype=int64) array([7], dtype=int64) array([8], dtype=int64) array([3, 9], dtype=int64)]",872.082598,6.000000,"[array([4, 2], dtype=int64) array([9, 5], dtype=int64) array([7, 3], dtype=int64) array([1], dtype=int64) array([6], dtype=int64) array([8], dtype=int64)]",922.314680,6,"[array([8, 2], dtype=int64) array([3, 9], dtype=int64) array([6], dtype=int64) array([1], dtype=int64) array([7, 4], dtype=int64) array([5], dtype=int64)]",1090.129161,6
1,small-10n-c50_70-d10_50,problemset/in/small-10n-c50_70-d10_50_nodes.parquet,problemset/in/small-10n-c50_70-d10_50_dm.parquet,55,"[array([2, 4, 5], dtype=int64) array([6], dtype=int64) array([3, 7], dtype=int64) array([8], dtype=int64) array([1, 9], dtype=int64)]",733.977081,5.000000,"[array([5, 4, 2], dtype=int64) array([9, 1], dtype=int64) array([7, 3], dtype=int64) array([6], dtype=int64) array([8], dtype=int64)]",733.977081,5,"[array([8, 2], dtype=int64) array([6], dtype=int64) array([3, 9, 7], dtype=int64) array([4, 5], dtype=int64) array([1], dtype=int64)]",945.011879,5
2,small-10n-c50_70-d10_50,problemset/in/small-10n-c50_70-d10_50_nodes.parquet,problemset/in/small-10n-c50_70-d10_50_dm.parquet,60,"[array([2, 4, 5], dtype=int64) array([6, 7], dtype=int64) array([3, 8], dtype=int64) array([1, 9], dtype=int64)]",651.901348,4.000000,"[array([5, 4, 2], dtype=int64) array([9, 1], dtype=int64) array([7, 6], dtype=int64) array([8, 3], dtype=int64)]",651.901348,4,"[array([8, 9], dtype=int64) array([6, 7], dtype=int64) array([5, 4, 2], dtype=int64) array([3, 1], dtype=int64)]",693.171486,4
3,small-10n-c50_70-d10_50,problemset/in/small-10n-c50_70-d10_50_nodes.parquet,problemset/in/small-10n-c50_70-d10_50_dm.parquet,65,"[array([2, 1, 3], dtype=int64) array([6, 7], dtype=int64) array([8], dtype=int64) array([5, 4, 9], dtype=int64)]",637.303323,4.000000,"[array([5, 4, 2], dtype=int64) array([9, 1], dtype=int64) array([7, 6], dtype=int64) array([8, 3], dtype=int64)]",651.901348,4,"[array([8, 3], dtype=int64) array([6, 7], dtype=int64) array([9, 1, 2], dtype=int64) array([4, 5], dtype=int64)]",708.420574,4
4,small-10n-c50_70-d10_50,problemset/in/small-10n-c50_70-d10_50_nodes.parquet,problemset/in/small-10n-c50_70-d10_50_dm.parquet,70,"[array([1, 2, 4], dtype=int64) array([6, 7], dtype=int64) array([8], dtype=int64) array([3, 9, 5], dtype=int64)]",623.374290,4.000000,"[array([5, 4, 2], dtype=int64) array([9, 1], dtype=int64) array([7, 6], dtype=int64) array([8, 3], dtype=int64)]",651.901348,4,"[array([8, 9, 2], dtype=int64) array([7, 5, 4], dtype=int64) array([3, 1], dtype=int64) array([6], dtype=int64)]",769.112343,4
5,small-10n-c50_60-d10_50,problemset/in/small-10n-c50_60-d10_50_nodes.parquet,problemset/in/small-10n-c50_60-d10_50_dm.parquet,50,"[array([3], dtype=int64) array([2, 4], dtype=int64) array([6], dtype=int64) array([5, 8], dtype=int64) array([1, 7, 9], dtype=int64)]",624.190751,5.000000,"[array([5, 1], dtype=int64) array([4, 9, 7], dtype=int64) array([8, 6], dtype=int64) array([2], dtype=int64) array([3], dtype=int64)]",669.586160,5,"[array([3], dtype=int64) array([2, 9], dtype=int64) array([8, 5], dtype=int64) array([6, 7, 4], dtype=int64) array([1], dtype=int64)]",716.098214,5
6,small-10n-c50_60-d10_50,problemset/in/small-10n-c50_60-d10_50_nodes.parquet,problemset/in/small-10n-c50_60-d10_50_dm.parquet,52,"[array([3], dtype=int64) array([2, 4], dtype=int64) array([6], dtype=int64) array([5, 8], dtype=int64) array([1, 7, 9], dtype=int64)]",624.190751,5.000000,"[array([5, 1], dtype=int64) array([4, 9, 7], dtype=int64) array([8, 6], dtype=int64) array([2], dtype=int64) array([3], dtype=int64)]",669.586160,5,"

## Archivos .parquet

In [ ]:
nodes = pd.read_parquet("/content/small-10n-c80_120-d10_50_nodes.parquet")
distance_matrix = pd.read_parquet("/content/small-10n-c80_120-d10_50_dm.parquet")

In [ ]:
nodes

,x,y,demand
0,50,50,0
1,69,56,33
2,74,19,45
3,57,36,49
4,45,67,25
5,23,82,41
6,46,63,41
7,69,12,15
8,61,21,15
9,78,54,15


In [ ]:
distance_matrix

,0,1,2,3,4,5,6,7,8,9
0,0.000000,24.186773,15.264338,9.848858,40.816663,65.946948,36.891733,20.223748,12.083046,25.059928
1,24.186773,0.000000,37.336309,23.323808,26.400758,52.839379,24.041631,44.000000,35.902646,9.219544
2,15.264338,37.336309,0.000000,24.041631,56.080300,81.055537,52.153619,8.602325,13.152946,35.227830
3,9.848858,23.323808,24.041631,0.000000,33.241540,57.201399,29.154759,26.832816,15.524175,27.658633
4,40.816663,26.400758,56.080300,33.241540,0.000000,26.627054,4.123106,60.008333,48.703183,35.468296
5,65.946948,52.839379,81.055537,57.201399,26.627054,0.000000,29.832868,83.761566,71.867934,61.717096
6,36.891733,24.041631,52.153619,29.154759,4.123106,29.832868,0.000000,55.946403,44.598206,33.241540
7,20.223748,44.000000,8.602325,26.832816,60.008333,83.761566,55.946403,0.000000,12.041595,42.953463
8,12.083046,35.902646,13.152946,15.524175,48.703183,71.867934,44.598206,12.041595,0.000000,37.121422
9,25.059928,9.219544,35.227830,27.658633,35.468296,61.717096,33.241540,42.953463,37.121422,0.000000


# Clustering

## Pseudocódigo



```
# INPUT:
    X = [x₁, x₂, ..., xₙ]       // coordenadas de los puntos (clientes)
    d = [d₁, d₂, ..., dₙ]       // demanda de cada cliente
    t = [t₁, t₂, ..., t_K]     // capacidad máxima de cada clúster (vehículo)
    K                          // número de clústeres (vehículos)
    max_iter                   // número máximo de iteraciones

OUTPUT:
    C = {C₁, C₂, ..., C_K}     // conjunto de clústeres (listas de índices de puntos)
                                  - Asignación de clientes a clústeres respetando su capacidad

ALGORITMO:

1. Inicializar los centroides p[0] = {c₁, c₂, ..., c_K} seleccionando K puntos aleatorios de X

2. num_iter = 0

3. Mientras num_iter < max_iter:
    4. num_iter += 1
    5. C = {C₁=[], C₂=[], ..., C_K=[]}  // vaciar asignaciones de clústeres
    6. data = combinar(X, t, d)         // array de tuplas [xᵢ, tᵢ, dᵢ]
    7. ordenar data en orden decreciente según dᵢ
    8. separar X', t', d' de data
    
    // Asignación
    9. Para i = 0 hasta n-1:
        10. x = X'[i]
        11. mejor_k = -1
        12. menor_dist = ∞
        13. tabu = []
        
        14. Mientras True:
            15. Para k = 0 hasta K-1:
                16. Si k ∉ tabu:
                    17. centroide = p[num_iter - 1][k]
                    18. dist = distancia(x, centroide)
                    19. Si dist < menor_dist:
                        20. menor_dist = dist
                        21. mejor_k = k
            22. Si suma_demandas(C[mejor_k]) + d'[i] ≤ t'[mejor_k]:
                23. C[mejor_k].agregar(i)
                24. salir del bucle
            25. Sino:
                26. agregar mejor_k a tabu
                27. Si todos los clústeres están en tabu:
                    28. asignar x a un nuevo clúster (opcional)
                    29. salir del bucle

    // Actualización de centroides
    30. Para k = 0 hasta K-1:
        31. Si C[k] no está vacío:
            32. p[num_iter][k] = media(X'[i] para i ∈ C[k])

FIN del ciclo

32. Retornar C

```



## Implementación

In [ ]:
class CKmeans:
    def __init__(self, K: int, max_iter: int = 100, random_state: int = 42):
        self.K = K
        self.max_iter = max_iter
        self.random_state = random_state
        self.centroids_history = []

    def fit(self, X: np.ndarray, d: List[float], t: List[float]) -> List[List[int]]:
        n = len(X)
        np.random.seed(self.random_state)
        random.seed(self.random_state)

        init_indices = np.random.choice(n, self.K, replace=False)
        centroids = X[init_indices]
        self.centroids_history.append(centroids.copy())

        for iteration in range(self.max_iter):
            C = [[] for _ in range(self.K)]
            carga_actual = [0.0 for _ in range(self.K)]

            data = list(zip(X, d, range(n)))
            data.sort(key=lambda x: -x[1])  # ordenar por demanda

            for x_i, demanda_i, idx in data:
                mejor_k = -1
                menor_dist = float('inf')
                tabu = set()

                while True:
                    for k in range(self.K):
                        if k in tabu:
                            continue
                        dist = np.linalg.norm(x_i - centroids[k])
                        if dist < menor_dist:
                            menor_dist = dist
                            mejor_k = k

                    if carga_actual[mejor_k] + demanda_i <= t[mejor_k]:
                        C[mejor_k].append(idx)
                        carga_actual[mejor_k] += demanda_i
                        break
                    else:
                        tabu.add(mejor_k)
                        if len(tabu) == self.K:
                            break

            new_centroids = np.zeros_like(centroids)
            for k in range(self.K):
                if len(C[k]) > 0:
                    new_centroids[k] = np.mean(X[C[k]], axis=0)
                else:
                    new_centroids[k] = X[np.random.choice(n)]

            centroids = new_centroids
            self.centroids_history.append(centroids.copy())

        return C

    def get_centroids_history(self) -> List[np.ndarray]:
        return self.centroids_history


class KmeansHeuristic:
    @staticmethod
    def estimate_optimal_k(X: np.ndarray, d: List[float], t: List[float]) -> int:
        n = len(X)
        total_demand = sum(d)
        avg_capacity = np.mean(t)

        # Heurística 1: Capacidad vs demanda
        k_capacity = max(1, math.ceil(total_demand / avg_capacity))

        # Heurística 2: Tamaño del problema
        if n <= 10:
            k_size = min(2, n)
        elif n <= 50:
            k_size = min(math.ceil(n / 8), 8)
        elif n <= 100:
            k_size = min(math.ceil(n / 12), 12)
        elif n <= 500:
            k_size = min(math.ceil(n / 20), 20)
        else:
            k_size = min(math.ceil(math.sqrt(n)), 30)

        # Promedio ponderado de ambas
        k_weighted = 0.6 * k_capacity + 0.4 * k_size
        k_optimal = max(1, min(round(k_weighted), n, len(t)))

        return k_optimal

    @staticmethod
    def get_k_range(X: np.ndarray, d: List[float], t: List[float]) -> Tuple[int, int]:
        k_optimal = KmeansHeuristic.estimate_optimal_k(X, d, t)
        k_min = max(1, k_optimal - 2)
        k_max = min(len(X), len(t), k_optimal + 3)
        return k_min, k_max

# CKmeans_ACO_VRP

In [ ]:
class CKmeans_ACO_VRP:
    def __init__(self,
                 K: int = None,  # Ahora K es opcional
                 max_iter_kmeans: int = 100,
                 num_ants: int = 10,
                 num_iterations_aco: int = 100,
                 alpha: float = 1.0,
                 beta: float = 2.0,
                 evaporation_rate: float = 0.1,
                 Q: float = 100,
                 vehicle_capacity: float = 100,
                 random_state: int = 42,
                 use_heuristic_k: bool = True):  # Nuevo parámetro

        self.K = K
        self.max_iter_kmeans = max_iter_kmeans
        self.num_ants = num_ants
        self.num_iterations_aco = num_iterations_aco
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.Q = Q
        self.vehicle_capacity = vehicle_capacity
        self.random_state = random_state
        self.use_heuristic_k = use_heuristic_k

        # Se inicializará después de determinar K
        self.ckmeans = None
        self.aco_vrp = ACO_VRP(num_ants, num_iterations_aco, alpha, beta,
                               evaporation_rate, Q, vehicle_capacity, random_state)

        self.clusters = []
        self.cluster_routes = []
        self.total_distance = 0
        self.total_vehicles = 0

    def solve(self, nodes_df, distance_matrix_df) -> dict:
        # Preparar datos para la heurística
        coordinates = nodes_df[['x', 'y']].values
        demands = nodes_df['demand'].tolist()

        # Determinar K si no se especificó o si se quiere usar heurística
        if self.K is None or self.use_heuristic_k:
            # Estimar capacidades basándose en la capacidad del vehículo
            max_possible_k = min(len(nodes_df) - 1, 30)  # Límite razonable
            estimated_capacities = [self.vehicle_capacity * 3 for _ in range(max_possible_k)]

            optimal_k = KmeansHeuristic.estimate_optimal_k(coordinates, demands, estimated_capacities)
            self.K = optimal_k
            print(f"K óptimo determinado por heurística: {self.K}")
        else:
            print(f"Usando K especificado: {self.K}")

        # Inicializar CKmeans con el K determinado
        self.ckmeans = CKmeans(self.K, self.max_iter_kmeans, self.random_state)

        print("Iniciando clustering con CKmeans...")
        capacities = [self.vehicle_capacity * 3 for _ in range(self.K)]
        self.clusters = self.ckmeans.fit(coordinates, demands, capacities)
        print(f"Clustering completado. {len([c for c in self.clusters if c])} clusters con datos.")

        all_routes = []
        total_distance = 0
        total_vehicles = 0

        for i, cluster_indices in enumerate(self.clusters):
            if not cluster_indices:
                continue

            print(f"Procesando cluster {i + 1} con {len(cluster_indices)} clientes...")

            cluster_nodes = self._create_cluster_subset(nodes_df, cluster_indices)
            cluster_distance_matrix = self._create_cluster_distance_matrix(distance_matrix_df, cluster_indices)

            cluster_routes, cluster_distance, num_vehicles = self.aco_vrp.solve(
                cluster_nodes, cluster_distance_matrix)

            mapped_routes = self._map_routes_to_global(cluster_routes, cluster_indices)

            all_routes.extend(mapped_routes)
            total_distance += cluster_distance
            total_vehicles += num_vehicles

            print(f"  Cluster {i + 1}: {num_vehicles} vehículos, distancia: {cluster_distance:.2f}")

        self.cluster_routes = all_routes
        self.total_distance = total_distance
        self.total_vehicles = total_vehicles

        print("\nRESULTADO FINAL:")
        print(f"   Total vehículos: {total_vehicles}")
        print(f"   Distancia total: {total_distance:.2f}")

        return {
            'routes': all_routes,
            'total_distance': total_distance,
            'total_vehicles': total_vehicles,
            'clusters': self.clusters,
            'cluster_info': self._get_cluster_info(nodes_df),
            'optimal_k': self.K  # Agregar K usado
        }

    def _create_cluster_subset(self, nodes_df, cluster_indices: List[int]):
        subset_indices = [0] + cluster_indices
        cluster_nodes = nodes_df.iloc[subset_indices].copy()
        cluster_nodes.reset_index(drop=True, inplace=True)
        return cluster_nodes

    def _create_cluster_distance_matrix(self, distance_matrix_df, cluster_indices: List[int]):
        subset_indices = [0] + cluster_indices
        cluster_matrix = distance_matrix_df.iloc[subset_indices, subset_indices].copy()
        cluster_matrix.reset_index(drop=True, inplace=True)
        cluster_matrix.columns = range(len(cluster_matrix))
        return cluster_matrix

    def _map_routes_to_global(self, local_routes: List[List[int]], cluster_indices: List[int]) -> List[List[int]]:
        index_mapping = {0: 0}
        for local_idx, global_idx in enumerate(cluster_indices, 1):
            index_mapping[local_idx] = global_idx

        return [[index_mapping[i] for i in route] for route in local_routes]

    def _get_cluster_info(self, nodes_df) -> List[dict]:
        cluster_info = []
        for i, cluster_indices in enumerate(self.clusters):
            if cluster_indices:
                cluster_demands = nodes_df.iloc[cluster_indices]['demand'].values
                cluster_info.append({
                    'cluster_id': i,
                    'size': len(cluster_indices),
                    'total_demand': float(np.sum(cluster_demands)),
                    'avg_demand': float(np.mean(cluster_demands)),
                    'indices': cluster_indices
                })
        return cluster_info

    def get_results_summary(self) -> str:
        resumen = f"""
=== RESUMEN CKmeans + ACO_VRP ===
K óptimo usado: {self.K}
Clusters creados: {len([c for c in self.clusters if c])}
Total vehículos: {self.total_vehicles}
Distancia total: {self.total_distance:.2f}
Vehículos por cluster: {[len([r for r in self.cluster_routes if any(idx in r for idx in cluster)]) for cluster in self.clusters if cluster]}
"""
        return resumen

    def analyze_clustering_quality(self, nodes_df) -> dict:
        """Analiza la calidad del clustering realizado"""
        coordinates = nodes_df[['x', 'y']].values
        demands = nodes_df['demand'].tolist()
        capacities = [self.vehicle_capacity * 3 for _ in range(self.K)]

        analysis = KmeansHeuristic.analyze_problem_characteristics(coordinates, demands, capacities)

        # Agregar métricas específicas del clustering realizado
        cluster_sizes = [len(cluster) for cluster in self.clusters if cluster]
        cluster_demands = []

        for cluster_indices in self.clusters:
            if cluster_indices:
                cluster_demand = sum(nodes_df.iloc[cluster_indices]['demand'])
                cluster_demands.append(cluster_demand)

        analysis.update({
            'actual_clusters_used': len([c for c in self.clusters if c]),
            'cluster_sizes': cluster_sizes,
            'cluster_demands': cluster_demands,
            'avg_cluster_size': np.mean(cluster_sizes) if cluster_sizes else 0,
            'cluster_size_std': np.std(cluster_sizes) if cluster_sizes else 0,
            'avg_cluster_demand': np.mean(cluster_demands) if cluster_demands else 0,
            'cluster_demand_std': np.std(cluster_demands) if cluster_demands else 0
        })

        return analysis

In [ ]:
class ACO_VRP:
    def __init__(self, num_ants, num_iterations, alpha, beta,
                 evaporation_rate, Q, vehicle_capacity, random_state=42):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.Q = Q
        self.vehicle_capacity = vehicle_capacity
        self.random_state = random_state
        np.random.seed(self.random_state)

    def _initialize(self, nodes_df, distance_matrix_df):
        self.nodes_df = nodes_df
        self.demands = nodes_df['demand'].values
        self.n = len(nodes_df)

        self.sub_dist = distance_matrix_df.values
        self.visibility = np.where(self.sub_dist > 0, 1.0 / self.sub_dist, 0.0)

        # Inicializar feromonas a un valor positivo constante
        self.pheromone = np.ones((self.n, self.n))

    def _construct_ant_solution(self):
        routes = []
        total_distance = 0.0
        unvisited = set(range(1, self.n))  # El nodo 0 es el depósito

        while unvisited:
            route = [0]
            current_load = 0.0
            current_node = 0

            while True:
                candidates = [
                    j for j in unvisited if current_load + self.demands[j] <= self.vehicle_capacity
                ]

                if not candidates:
                    break

                probs = []
                for j in candidates:
                    tau = self.pheromone[current_node][j] ** self.alpha
                    eta = self.visibility[current_node][j] ** self.beta
                    probs.append(tau * eta)

                probs = np.array(probs)
                probs_sum = probs.sum()
                if probs_sum == 0:
                    next_node = np.random.choice(candidates)
                else:
                    probs /= probs_sum
                    next_node = np.random.choice(candidates, p=probs)

                route.append(next_node)
                current_load += self.demands[next_node]
                unvisited.remove(next_node)
                current_node = next_node

            route.append(0)  # Regreso al depósito
            routes.append(route)

            # Calcular distancia total del recorrido
            distance = sum(self.sub_dist[route[i]][route[i + 1]] for i in range(len(route) - 1))
            total_distance += distance

        return routes, total_distance

    def _update_pheromones(self, all_routes, all_distances):
        # Evaporación
        self.pheromone *= (1 - self.evaporation_rate)

        # Depósito de feromonas por cada hormiga
        for routes, dist in zip(all_routes, all_distances):
            if dist == 0:  # Prevención contra división por cero
                continue
            deposit = self.Q / dist
            for route in routes:
                for i in range(len(route) - 1):
                    a, b = route[i], route[i + 1]
                    self.pheromone[a][b] += deposit
                    self.pheromone[b][a] += deposit  # Simetría

    def solve(self, nodes_df, distance_matrix_df):
        self._initialize(nodes_df, distance_matrix_df)

        best_distance = float('inf')
        best_routes = []

        for iteration in range(self.num_iterations):
            all_routes = []
            all_distances = []

            for _ in range(self.num_ants):
                routes, distance = self._construct_ant_solution()
                all_routes.append(routes)
                all_distances.append(distance)

            self._update_pheromones(all_routes, all_distances)

            # Actualizar la mejor solución encontrada
            min_idx = np.argmin(all_distances)
            if all_distances[min_idx] < best_distance:
                best_distance = all_distances[min_idx]
                best_routes = all_routes[min_idx]

        return best_routes, best_distance, len(best_routes)

# Validación

In [ ]:
    # Cargar datos
    nodos_path = '/content/small-10n-c50_60-d10_50_dm.parquet'
    dm_path = '/content/small-10n-c50_60-d10_50_nodes.parquet'

    df_nodos = pd.read_parquet(nodos_path)
    df_nodos.columns = [c.strip().lower() for c in df_nodos.columns]

    matriz_distancias = pd.read_parquet(dm_path).values
    matriz_df = pd.DataFrame(matriz_distancias, index=df_nodos.index, columns=df_nodos.index)

    # Usar la versión híbrida con heurística automática de K
    solver = CKmeans_ACO_VRP(
        num_ants=10,
        num_iterations_aco=80,
        alpha=1.0,
        beta=2.0,
        evaporation_rate=0.1,
        Q=100,
        # vehicle_capacity=56,
        use_heuristic_k=True,  # K automático
        random_state=42
    )

    resultado = solver.solve(df_nodos, matriz_df)

    # Extraer resultados
    routes = resultado['routes']
    total_distance = resultado['total_distance']
    num_vehicles = resultado['total_vehicles']

    # Mostrar resultados
    print("=== RESULTADO CKmeans + ACO_VRP ===")
    print(f"K óptimo usado: {resultado['optimal_k']}")
    print(f"Total vehículos: {num_vehicles}")
    print(f"Distancia total: {total_distance:.7f}")
    print("Rutas:")
    for i, route in enumerate(routes, 1):
        route_clean = [int(node) for node in route]
        print(f"  Vehículo {i}: {route_clean}")

    # Mostrar información de clusters
    print("\nInformación de clusters:")
    for cluster_info in resultado['cluster_info']:
        print(f"  Cluster {cluster_info['cluster_id']}: {cluster_info['size']} nodos, "
              f"demanda total: {cluster_info['total_demand']:.2f}")

    # Visualizar rutas con matplotlib
    import matplotlib.pyplot as plt

    plt.figure(figsize=(12, 8))
    colors = plt.cm.get_cmap('tab20', num_vehicles)

    for i, route in enumerate(routes):
        route_nodes = df_nodos.loc[route]
        plt.plot(route_nodes['x'], route_nodes['y'], marker='o',
                label=f'Vehículo {i+1}', color=colors(i), linewidth=2, markersize=6)

    # Dibujar el depósito
    if 0 in df_nodos.index:
        depot = df_nodos.loc[0]
        plt.scatter(depot['x'], depot['y'], c='red', s=200, marker='s',
                   label='Depósito', edgecolor='black', linewidth=2, zorder=5)

    plt.title(f"Rutas calculadas por CKmeans + ACO_VRP (K={resultado['optimal_k']})")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()